# U.S. Patent Phrase to Phrase Matching

In [19]:
import pandas as pd
import numpy as np
import scipy as sp
import nltk 
from nltk.tokenize import sent_tokenize #paragraftan cümleleri ayırıp liste haline getirir.
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline


In [11]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mehmetburakerkan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
train = pd.read_csv("train.csv")
train.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [8]:
train.head()

,id,anchor,target,context
0,37d61fd2272659b1,abatement,abatement of pollution,A47
1,7b9652b17b68b7a4,abatement,act of abating,A47
2,36d72442aefd8232,abatement,active catalyst,A47
3,5296b0c19e1ce60e,abatement,eliminating process,A47
4,54c1e3b9184cb5b6,abatement,forest region,A47


In [30]:
train.target.value_counts()

composition                    24
data                           22
metal                          22
motor                          22
assembly                       21
                               ..
switching switch over valve     1
switching switch off valve      1
switching over valve            1
switching off valve             1
wooden substrate                1
Name: target, Length: 29340, dtype: int64

In [9]:
len(train)

36473

In [4]:
train.isnull().sum()

id         0
anchor     0
target     0
context    0
score      0
dtype: int64

In [5]:
y = train.score

In [38]:
y = pd.DataFrame(y)

In [7]:
del train["score"]

In [3]:
test = pd.read_csv("test.csv")
test.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [25]:
vect = CountVectorizer()

anchor = vect.fit_transform(train.anchor)
train_anchor = pd.DataFrame(anchor.toarray(), columns=vect.get_feature_names())


In [26]:
anchor

<36473x1113 sparse matrix of type '<class 'numpy.int64'>'
	with 79434 stored elements in Compressed Sparse Row format>

In [27]:
train_anchor

,abatement,abnormal,abrasion,absorbent,absorption,acan,acceleration,accept,accidental,achieve,...,whose,width,wire,wiring,with,within,wood,working,workpiece,writing
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
36469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
36470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
36471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [28]:
target = vect.fit_transform(train.target)
train_target = pd.DataFrame(target.toarray(), columns=vect.get_feature_names())
train_target

,10,111444,12,120,124,12cao7al2o3,132,1333,133a466a,135,...,zero,zinc,zipper,zirconia,zolpidem,zone,zones,zoo,zoom,zoster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
context = train["context"]

In [32]:
context = pd.DataFrame(context)

In [34]:
context = pd.get_dummies(context,drop_first=True)

In [35]:
context

,context_A21,context_A22,context_A23,context_A24,context_A41,context_A43,context_A44,context_A45,context_A46,context_A47,...,context_G09,context_G10,context_G11,context_G16,context_G21,context_H01,context_H02,context_H03,context_H04,context_H05
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
d1 = train_anchor.merge(train_target,left_index=True, right_index=True)
hazır = d1.merge(context,left_index=True, right_index=True)
hazır

,abatement_x,abnormal_x,abrasion_x,absorbent_x,absorption_x,acan,acceleration_x,accept_x,accidental_x,achieve_x,...,context_G09,context_G10,context_G11,context_G16,context_G21,context_H01,context_H02,context_H03,context_H04,context_H05
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor().fit(hazır,y)

In [41]:
hazır.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Columns: 10100 entries, abatement_x to context_H05
dtypes: int64(9995), uint8(105)
memory usage: 2.7 GB


In [40]:
y

,score
0,0.50
1,0.75
2,0.25
3,0.50
4,0.00
...,...
36468,1.00
36469,0.50
36470,0.50
36471,0.75


In [43]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   score   36473 non-null  float64
dtypes: float64(1)
memory usage: 285.1 KB


In [ ]:
vect = CountVectorizer()

anchor = vect.fit_transform(test.anchor)
test_anchor = pd.DataFrame(anchor.toarray(), columns=vect.get_feature_names())

In [ ]:
target = vect.fit_transform(test.target)
test_target = pd.DataFrame(target.toarray(), columns=vect.get_feature_names())

In [ ]:
context_t = test["context"]
context_t = pd.DataFrame(context_t)

In [ ]:
context = pd.get_dummies(context,drop_first=True)

In [ ]:
d2 = test_anchor.merge(test_target,left_index=True, right_index=True)
hazır_2 = d2.merge(context_t,left_index=True, right_index=True)
hazır_2

In [ ]:
sonuc = model.predict(hazır2)